**<h1><center>Laboratorio 11: LLM y Agentes Autónomos 🤖</center></h1>**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- Profesor: Ignacio Meza, Sebastian Tinoco
- Auxiliar: Catherine Benavides, Consuelo Rojas
- Ayudante: Eduardo Moya, Nicolás Ojeda

### **Equipo:**

- Nombre de alumno 1: Manuel Zamorano
- Nombre de alumno 2: Javier Urrutia

**SUPER IMPORTANTE** - notebooks sin nombre no serán revisados

### **Link de repositorio de GitHub:** `https://github.com/JUrrutia10/Laboratorios-MDS`

### **Indice**

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## **Temas a tratar**

- Implementación de modelos de LLM y Reinforcement Learning.
- Utilización e implementación de agentes.

## **Reglas:**

- **Grupos de 2 personas**
- Fecha de entrega: 7 días desde la publicación, 3 días de atraso con 1 punto de descuento c/u.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- Prohibidas las copias. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
Pueden usar cualquer material del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Generar un modelo LLM generativo interactivo.
- Entrenar un modelo de Reinforce Learning.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Large Language Models (4.0 puntos)**

<p align="center">
  <img src="https://trestristescriticos.com/wp-content/uploads/2021/07/telefono-gratuito-cinesur.jpg" width="350">
</p>

Joaquín no es un aficionado del cine, pero a principios de año, se propuso ver más peliculas para poder tener más temas de conversación con sus amigos y familia. Sin embargo, ya es junio y Joaquín no ha visto ninguna pelicula nueva o relevante de las que tenía en su lista y su reunión familiar bi-anual se acerca y necesita la mayor información que pueda recopilar de dichas peliculas sin tener que verlas.

Para esto, usted con su compañerx, tendrá que crear una aplicación utilizando LangChain.


---



**Instalación de librerías**

Para la creación de la aplicación, se utilizara un modelo de lenguaje (LLM) ofrecido gratuitamente por Google.

Para ello, se utilizará la API de Gemini, por lo que si no tienen acceso, se pueden crear una cuenta en el siguiente [enlace a Google AI](https://ai.google.dev/). Ahí, ir a la pestaña superior y seleccione la opción que dice ``Gemini API``.

<img src='https://gitlab.com/imezadelajara/datos_clase_7_mds7202/-/raw/main/misc_images/Screenshot_2024-06-13_at_12.42.32_PM.png' width='450' />

Luego, seleccione el botón que dice ``Get API key in Google AI Studio`` y hacer click en ``Crear clave de API`` para generar la llave con la que se podrá consultar al modelo de lenguaje.

<img src='https://gitlab.com/imezadelajara/datos_clase_7_mds7202/-/raw/main/misc_images/Screenshot_2024-06-13_at_12.45.10_PM.png?ref_type=heads' width='450' />

**Importante:** Debido a las restricciones de esta API, lo ideal es utilizar la llave a la API de manera personal.


Para mayor información sobre **LangChain**, pueden revisar la documentación en el [presente enlace](https://python.langchain.com/v0.2/docs/tutorials/summarization/ ).

In [1]:
%%capture
!pip install langchain
!pip install langchain_google_genai
!pip install langchain-community
!pip install langchain-experimental
!pip install sentence-transformers
!pip install faiss-cpu

In [134]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

os.environ["GOOGLE_API_KEY"] = #aqui va el key de api
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

### **1.1 Carga y limpieza (0.5 puntos)**

Para iniciar su titanica tarea de enseñarle a Joaquín sobre las mejores peliculas del último tiempo, tiene que revisar los script de las siguientes 3 peliculas:
* Dune 2
* Under Paris
* Joker
Debe encontrar un patrón y obtener solamente el guión de las películas. Para ello se recomienda utilizar métodos de búsqueda y reemplazo que tienen los ``string`` en Python. Adicionalmente, puede usar filtros de expresiones regulares.

Posterior a la limpieza de los guiones, debe considerar que el patrón se repite y es generalizable.


In [18]:
# Scripts de peliculas
dune2_script="https://scrapsfromtheloft.com/movies/dune-part-two-2024-transcript/"
underparis_script="https://scrapsfromtheloft.com/movies/under-paris-2024-transcript/"
joker_script="https://scrapsfromtheloft.com/movies/joker-2019-transcript/"

In [19]:
import re
from langchain.document_loaders import WebBaseLoader

def load_website_data(url):
    """
    Carga el contenido de un sitio web dado una URL.
    """
    loader = WebBaseLoader(url)
    data = loader.load()
    website_data = data[0].page_content if data else ""
    return website_data

def clean_script(text, start_marker="["):
    """
    Limpia el texto para obtener solo el guión de la película.
    """
    # Encontrar el inicio del guión (usando un marcador específico)
    start_pattern = re.compile(re.escape(start_marker))
    start_marker_match = start_pattern.search(text)
    start_index = start_marker_match.start() if start_marker_match else 0

    # Eliminar contenido innecesario después del guión
    end_marker = re.search(r"\bSHARE THIS ARTICLE\b", text, re.IGNORECASE)
    end_index = end_marker.start() if end_marker else len(text)

    # Recortar el texto para obtener solo el guión
    text_cleaned = text[start_index:end_index].strip()

    return text_cleaned

In [22]:
# Cargar el contenido de los guiones
dune2 = load_website_data(dune2_script)
underparis = load_website_data(underparis_script)
joker = load_website_data(joker_script)

# Limpiar el texto para obtener solo el guión
dune2_script_clean = clean_script(dune2)
underparis_script_clean = clean_script(underparis)
joker_script_clean = clean_script(joker)

# Imprimir una muestra del guión limpio de Dune 2
print(dune2_script_clean[:500])  # Mostrar los primeros 500 caracteres del guión limpio para ver el resultado

[male voice in alien language] Power over Spice is power over all
[suspenseful music playing]
[Irulan] Imperial Diary. Year 10,191. Third comment. The battle for Arrakis took everyone by surprise. There were no witnesses.
[somber music playing]
The Harkonnen operation was perpetrated overnight, without warning or declaration of war. By morning, the Atreides were no more. All died in the dark.
And the Emperor said… nothing. Since that night, my father has not been the same. Nor have I. His inacti


In [23]:
print(dune2_script_clean[-500:])

 heavily]
[stomps]
[inhales deeply]
My Lord, the Great Houses have answered.
They refuse to honor your ascendancy.
We await your orders, Lisan al-Gaib.
Lead them to Paradise.
[female Fremen] Lisan al-Gaib!
[all chanting] Lisan al-Gaib! Lisan al-Gaib!
[speaking Chakobsa]
[all chanting] Lisan al-Gaib! Lisan al-Gaib!
[Alia] What is happening, Mother?
Your brother attacks the Great Houses.
The Holy War begins.
[breathing heavily]












More:

Denis Villeneuve, Dune: Part Two, Movie Transcripts


### **1.2 Aplicación (3.5 puntos)**

Luego de limpiar los guiones, es posible generar la aplicaicón deseada con el LLM. Esta aplicación tiene que ser capaz de realizar las siguientes tareas.

1. Utilizando una plantilla sobre el nombre del archivo o la URL, identifique el supuesto nombre de la película.

2. Genere un resumen en español de la película y una nota evaluativa sobre la misma. El resumen debe tener entre 3 a 5 párrafos. Además, obtener una evaluación de la película con una calificación del 1 al 10, utilizando una LLM y el contexto entregado

#### **1.2.1 Título de la película (0.5 puntos)**

Para obtener el título, utilicé la siguiente plantilla:
```
 template = """
  What is the movie that appears in the description of this file or url?
  You only give me the movie name, nothing more.
  document/url: {script_path_url}
  """
```



In [32]:
# Plantilla para obtener el título de la película
template = """
What is the movie that appears in the description of this file or url?
You only give me the movie name, nothing more.
document/url: {script_path_url}
"""

def get_movie_title(script_path_url):
    """
    Utiliza el modelo de lenguaje para obtener el título de la película
    basado en la plantilla y la URL del guión.
    """
    # Completar la plantilla con la URL del guión
    prompt = template.format(script_path_url=script_path_url)

    # Generar la respuesta utilizando el modelo de lenguaje
    response = llm.invoke(prompt)

    # Limpiar y obtener el título de la película de la respuesta generada
    movie_title = response.content

    return movie_title

In [33]:
movie_title = get_movie_title(dune2_script)
print("Movie title:", movie_title)

Movie title: Dune: Part Two 



In [34]:
movie_title = get_movie_title(underparis_script)
print("Movie title:", movie_title)

Movie title: Under Paris 



In [35]:
movie_title = get_movie_title(joker_script)
print("Movie title:", movie_title)

Movie title: Joker 



#### **1.2.2 Resumen (1.0 puntos)**

Como se vió en clases, las LLM no pueden manejar cadenas de texto muy largas, esto es debido a que, dependiendo de su naturaleza, solo manejan ventanas de contexto que estan asociadas a caracteristicas de la red y del entrenamiento utilizado.

Por ello, es altamente importante que si se desea hacer un resumen del texto, este se haga realizando un tipo de map/reduce sobre el texto. De manera que en cada una de las iteraciones se vaya disminuyendo el tamaño del texto, pero hay que tener cuidado con que le modelo vaya guardando el contexto de escenas previas.

In [36]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [154]:
# crear templates

map_template_summary = """
Genera un resumen de la película basado en el guion proporcionado:
{script}
"""

reduce_template_summary = """
Reduce el resumen anterior para hacerlo más conciso:
{script_resume}
"""

# Respuesta final del resumen
answer_summary = """
Resumen de la película:
{summary}
"""

In [157]:
def map_reduce_text(script_imput, map_template, reduce_template):


    
    document_prompt = PromptTemplate(
    input_variables=["script"],
    template=map_template
    )
    document_variable_name = "script_resume"

    # The prompt here should take as an input variable the
    # `document_variable_name`
    prompt = PromptTemplate.from_template(
        template=reduce_template
    )
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_prompt=document_prompt,
        document_variable_name=document_variable_name
    )
    
    # Map/Reduce
    """
    Utilizar MapReduceDocumentsChain
    """
    reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=chain,
    )
    MRchain = MapReduceDocumentsChain(
        llm_chain=llm_chain,
        reduce_documents_chain=reduce_documents_chain,
    )

    # Text splitter
    """
    Usar RecursiveCharacterTextSplitter
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_script = text_splitter.split_text(script_imput)

    # Resultado
    result = MRchain({"input_documents": split_script})

    return result["output_text"]

In [158]:
# Ejemplo de uso
script = "This is a sample movie script..."
map_template_summary = """
Genera un resumen de la película basado en el guion proporcionado:
{script}
"""
reduce_template_summary = """
Reduce el resumen anterior para hacerlo más conciso:
{script_resume}
"""


summary = map_reduce_text(script, map_template_summary, reduce_template_summary)
print(f"Resumen de la película:\n{summary}")

AttributeError: 'str' object has no attribute 'page_content'

In [139]:
def map_reduce_text(script, map_template, reduce_template):

    
    document_prompt = PromptTemplate(
    input_variables=["script"],
     template=map_template
    )
    document_variable_name = "context"
    # The prompt here should take as an input variable the
    # `document_variable_name`
    prompt = PromptTemplate.from_template(
        template=reduce_template
    )
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    # We now define how to combine these summaries
    reduce_prompt = PromptTemplate.from_template(
        template=reduce_template
    )
    reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_llm_chain,
        document_prompt=document_prompt,
        document_variable_name=document_variable_name
    )
    reduce_documents_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_documents_chain,
    )
    chain = MapReduceDocumentsChain(
        llm_chain=llm_chain,
        reduce_documents_chain=reduce_documents_chain,
    )

    # Invocar el proceso de Map/Reduce sobre el texto
    result = chain({"input_documents": script})

    return result["output_text"]

In [ ]:
#No cambiar función

def map_reduce_text(script, map_template, reduce_template):
    # Crear el prompt para el Map
    map_prompt = PromptTemplate(input_variables=["script"],template=map_template)
    map_chain = LLMChain(prompt=map_prompt)
    

    # Crear el prompt para el Reduce
    reduce_prompt = PromptTemplate(template=reduce_template, input_variables=["script_resume"])
    reduce_chain = LLMChain(reduce_prompt)

    # Combinar y reducir los documentos usando StuffDocumentsChain y ReduceDocumentsChain
    combine_chain = StuffDocumentsChain(ReduceDocumentsChain(reduce_chain), max_tokens=4000)

    # Utilizar MapReduceDocumentsChain
    map_reduce_chain = MapReduceDocumentsChain(map_chain, combine_chain)

    # Dividir el texto utilizando RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter()
    split_script = text_splitter(script)

    # Invocar el proceso de Map/Reduce sobre el texto
    result = map_reduce_chain.invoke(split_script)

    return result["output_text"]

In [131]:
def evaluate_movie(script, evaluation_template):
    # Crear el prompt para la evaluación
    evaluation_prompt = PromptTemplate(template=evaluation_template)
    evaluation_chain = LLMChain(prompt=evaluation_prompt,llm=llm)

    # Generar la evaluación utilizando la LLM
    evaluation_response = evaluation_chain.run({"script": script})
    
    return evaluation_response

In [132]:
# Plantilla para la evaluación de la película
evaluation_template = """
Evalúa la película en una escala del 1 al 10, basada en el guion proporcionado:
{script}, solo dando la nota final
"""

# Respuesta final de la evaluación
answer_evaluation = """
Evaluación de la película: {evaluation}
"""

In [140]:
# imprimir resumenes de películas.

# Generar resumen para Dune 2
dune2_summary = map_reduce_text(dune2_script_clean, map_template_summary, reduce_template_summary)
print(answer_summary.format(summary=dune2_summary))

# Evaluar películas
dune2_evaluation = evaluate_movie(dune2_script_clean, evaluation_template)
print(answer_evaluation.format(evaluation=dune2_evaluation))


KeyError: 'llm_chain'

Adicionalmente, Joaquín sabe que su primo favorito le gusta ``Dune: Part 2`` por lo que le gustaría tener mayor información al respecto, para ello realice las siguientes tareas:


3. Genere un gráfico que muestre los personajes de la película con más apariciones en la misma.
4. Genere una tabla en pandas con los 3 personajes que más aparecen, indicando el nombre del actor y su edad actual más uno (ojo edad + 1).
5. Cree una función que responda preguntas sobre la película basándose en la información del texto entregado (OJO: las preguntas y salidas deben ser en español). Luego, responda las siguientes preguntas:
* ¿Qué y quién es Lisan al-Gaib?
* ¿Qué personaje no cree en la profecía pero es parte de ella?
* ¿Cuál es el objetivo de Feyd-Rautha?
6. Utilizando el top 3 de personajes que más aparecen en la película, genere con el modelo LLM y utilizando el contexto del guion, las 6 estadísticas que demuestren las habilidades de los personajes: Intelligence, Strength, Charisma, Wisdom, Emotional Resilience, y Creativity.

#### **1.2.3 Personajes (0.5 puntos)**

En la siguiente sección, tiene que entregar un template de personajes y redicción

In [ ]:
map_template_characters = """
  """

reduce_template_characters = """
  """

answer_character_list = map_reduce_text(
    website_data_1,
    map_template_characters,
    reduce_template_characters
)


In [ ]:
import pandas as pd
import plotly.express as px
from itertools import count
import ast
import re

def plot_characters(answer_character_list):
  # Clear answer
  answer_character_list = # ...
  characters = #...

  # distil the characters output
  """
  Recomendación, utilizar un diccionario para ordenar los personajes
  """

  # Create dataframe
  """
  De diccionario a DataFrame
  """

  # Graficar datos

  #Retornar los personajes
  return


#### **1.2.4 Actores principales (0.75 puntos)**

Importante saber que el script **no** maneja información de los actores, por ello, es importante que nuestra LLM tenga acceso a internet, de manera de poder realizar búsquedas que nos ayuden a completar la información consultada.

Para esto, utilizaremos agentes combinados con react para realzar la consulta y asegurarnos de que la respuesta es correcta.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentType, initialize_agent

In [ ]:
# Key para realizar una busqueda
os.environ["SERPER_API_KEY"] = 'd63e62662ef63eb9e44ab133d191f7a99a0024a3'

In [ ]:
def get_actors_and_age(character):

  # Inicializar tools y agente.
  tools = # ...
  agent = # ...

  # Crear template de query
  query_template = """

  """

  # Crear prompt y usar agente para la búsqueda.

  # Retornar Nombre y Edad + 1

  return

**Explicar metodología utilizada**

#### **1.2.5 Personajes Stats (0.5 puntos)**

Esta parte es similar al punto 2. La clave esta en crear un buen prompting que nos permita generar las estadísticas basandonos en una búsqueda por map/reduce.

Tras la búsqueda, la idea es tener una función de Python que nos permita generar el gráfico deseado y tener el resumen de los personajes.


In [ ]:
import json

def map_reduce_text(script, character):
  # Map
  map_template = """
  Crear template, utilizar las palabras claves:
  Intelligence, Charisma, Strength, Wisdom, Emotional Resilience and Creativity.
  """

  # crear prompt y cadena
  map_template += template_complemt
  map_prompt = # ...
  map_chain = # ...

  # Reduce
  reduce_template = """
  Crear prompt de reducción.
  Reducir, dado el perfil, en escala del 1 al 10 las cualidades mencionadas
  """
  reduce_prompt = # ...
  reduce_chain = # ...

  # Reduce
  """
  Reducir y combinar los documentos con un máximo de 4000 tokens
  """

  # Map/Reduce
  """
  Uilizar MapReduceDocumentsChain
  """


  # Text splitter
  """
  Usar RecursiveCharacterTextSplitter
  """

  result = map_reduce_chain.invoke(split_script)
  return result["output_text"]


# Formato del perfil
def format_profile(answer_character_profile):
  """
  Crear un json con las caracteristicas y que retorne
  (final_profile, stats) del personaje
  """
  return (final_profile, stats)


In [ ]:
# Escriba su respuesta acá

In [ ]:
final_profile, stats = format_profile(answer_character_profile)

In [ ]:
print(final_profile)

In [ ]:
# Función para gráficar stats. No Tocar.

import plotly.graph_objects as go
import numpy as np

def plot_stats(stats, character_name="Paul Atreides"):
    base_stats = [
        "Intelligence", "Charisma", "Strength",
        "Wisdom", "Emotional Resilience", "Creativity"
    ]
    for stat in base_stats:
        if stat not in stats:
            stats[stat] = 0

    labels = list(stats.keys())
    stats_values = list(stats.values())
    stats_values += stats_values[:1]
    labels += labels[:1]

    # Plotly figure
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=stats_values,
        theta=labels,
        fill='toself',
        name=character_name
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max(stats_values)]
            )
        ),
        showlegend=False,
        title=character_name
    )

    return fig

fig = plot_stats(stats)

#### **Comentar (0.25 puntos)**
Explicar metodología y secuencia lógica de cada una de las respuestas. Además responda:

* ¿Qué otras tareas se podría realizar? De dos ejemplos con la metodología asociada.

* ¿Cual es la importancia de los prompt y como estos afectan al desempeño de los LLM?

* ¿Alguna de sus respuestas fue una 'alucinación'? ¿Por qué sucede esto?

## **2. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **2.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

Joaquín es fanático del Blackjack, por lo que en esta subsección implementarán métodos de RL y así generar una estrategia para que pueda ~~ir al casino a  hacerse millonario~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **2.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.


El objetivo del Blackjack  es obtener una mano con un valor total lo más cercano posible a 21 sin pasarse.

- Un estado en el Blackjack puede representarse mediante una tupla la cual tiene la información de la suma total de las cartas del jugador, la carta visible del dealer y un indicador que diga que si es que el jugador tiene un As que puede contar como 11 sin pasarse de 21. Por ende, los estados son las posibles combinaciones de estas variables.

- Las acciones disponibles para el jugador son hit (el jugador solicita una carta adicional) y stick (el jugador decide no pedir más cartas y se planta con su mano actual).

- Las recompensas en el juego están definidas de la siguiente manera:
     - `+1`: Si el jugador gana la partida.
     - `0`: Si la partida termina en empate.
     - `-1`: Si el jugador pierde la partida.

#### **2.1.2 Generando un Baseline (0.2 puntos)**

* Simule un escenario en donde se escojan acciones aleatorias. Repita esta
simulación 5000 veces y reporte el promedio y desviación de las recompensas.
* ¿Cómo calificaría el performance de esta política?
* ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
env.step(1)

(array([24,  8,  0]), -1.0, True, False, {})

In [ ]:
# Simulación ambiente con acciones random
num_episodes = 5000
rewards = []

for _ in range(num_episodes):
    observation, _ = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action = env.action_space.sample()  # Tomar una acción aleatoria
        observation, reward, done, truncated, info = env.step(action)
        episode_reward += reward
        done = done or truncated  # Considerar truncado como término del episodio
    rewards.append(episode_reward)

# Calculo del promedio y desviación estándar
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)

mean_reward, std_reward

(-0.3888, 0.8986849058485404)

###¿Cómo calificaría el performance de esta política?
El valor promedio negativo indica que la política de acciones aleatorias resulta en pérdidas para el jugador. En este caso, el promedio -0.3888 implica que el jugador pierde en promedio aproximadamente 0.39 unidades de recompensa por episodio. Por otro lado, la desviación estándar de 0.8987 muestra una variabilidad considerable en las recompensas obtenidas, indicando que los resultados pueden variar significativamente de un episodio a otro (debido a la aleatoridad). En algunos episodios, el jugador podría ganar o empatar, pero en muchos otros es probable que pierda.

Dado lo anterior, el rendimiento de la política aleatoria en Blackjack es pobre como se esperaba. Esto se debe a que este juego  tiene estrategias específicas que pueden aumentar las probabilidades de ganar o minimizar las pérdidas.

###¿Cómo podría interpretar las recompensas obtenidas?
Recordemos que cuanado la recompensa es -1, el jugador ha perdido la partida, una recompensa de 0 indica que la partida terminó en empate y que una recompensa de +1 indica que el jugador ha ganado la partida. Dado esto, la política de acciones aleatorias tiene una mayor tendencia a perder, como lo muestra el promedio negativo de las recompensas.



#### **2.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Crear el modelo DQN
model = DQN("MlpPolicy", env, verbose=1)

# Entrenar el modelo
model.learn(total_timesteps=100000)

# Guardar el modelo
model.save("dqn_blackjack")

Se han truncado las últimas 5000 líneas del flujo de salida.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.177    |
|    n_updates        | 24460    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.46     |
|    ep_rew_mean      | -0.02    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 63212    |
|    fps              | 293      |
|    time_elapsed     | 333      |
|    total_timesteps  | 97948    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.222    |
|    n_updates        | 24461    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.46     |
|    ep_rew_mean      | -0.03    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 63216

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -0.043, Std reward: 0.9555893469477357


#### **2.1.4 Evaluación de modelo (0.2 puntos)**

* Repita el ejercicio 2.1.2 pero utilizando el modelo entrenado.
* ¿Cómo es el performance de su agente?
* ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Evaluar la política entrenada
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5000)

print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

Mean reward: -0.0484, Std reward: 0.9512399486985395


La recompensa media del agente entrenado (-0.0484) es considerablemente mejor que la recompensa media del escenario baseline (-0.3888).
Esto sugiere que el agente entrenado con DQN ha aprendido una política que resulta en menores pérdidas en comparación con las acciones aleatorias. La mejora en la recompensa media indica que el agente está tomando decisiones más informadas y efectivas durante el juego, pero no necesariamente es lo óptimo (debido a su negatividad).

Por otro lado, la desviación estándar de las recompensas del agente entrenado (0.9512) es ligeramente mayor que la del escenario baseline (0.8987).
Una desviación estándar mayor implica una mayor variabilidad en las recompensas obtenidas. Aunque esto no es ideal, la mejora significativa en la recompensa media es un indicativo positivo del rendimiento del agente.

#### **2.1.5 Estudio de acciones (0.2 puntos)**

* Genere una función que reciba un estado y retorne la accion del agente.
* Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

  * Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
  * Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

* ¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
# Crear y envolver el entorno
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

# Definir la función para obtener la acción del agente
def get_action(state):
    state = np.array(state).flatten()  # Asegurarse de que el estado esté aplanado
    action, _states = model.predict(state, deterministic=True)
    return action

# Escenarios proporcionados
scenarios = [
    ((6, 7, False), "Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene un as"),
    ((19, 3, True), "Suma de cartas del agente es 19, dealer muestra un 3, agente tiene un as")
]

# Obtener y mostrar la acción para cada escenario
for state, description in scenarios:
    action = get_action(state)
    print(f"Escenario: {description}\nAcción elegida por el agente: {action}\n")

Escenario: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene un as
Acción elegida por el agente: 1

Escenario: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene un as
Acción elegida por el agente: 0



Para el primer escenario, la mayoría de las estrategias recomendarían pedir una carta porque la mano del jugador es muy débil. Incluso si pedir aumenta el riesgo de pasarse de 21, es la acción correcta en este caso para intentar mejorar la mano. La acción de pedir es coherente con las reglas generales del Blackjack para una mano débil contra una carta fuerte del crupier

Para el esceneario 2 la estrategia óptima es plantarse. Una mano de 19 es muy fuerte y no tiene sentido arriesgarse a pedir otra carta y potencialmente pasarse de 21. La acción de plantarse es coherente con las reglas del Blackjack para una mano blanda fuerte contra una carta débil del crupier.

### **2.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.


#### **2.2.1 Descripción de MDP (0.2 puntos)**


Comencemos preparando el ambiente:

In [ ]:
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

* Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.
* ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?
* En la preparación del ambiente se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

El ambiente LunarLander simula el aterrizaje de una nave lunar en la superficie de la luna. El objetivo del agente es controlar la nave para que aterrice suavemente en una zona de aterrizaje marcada, evitando colisiones con obstáculos y minimizando el uso de combustible.

El estado del MDP en LunarLander incluye la posición y velocidad de la nave lunar, así como la orientación y la velocidad angular.
También se incluyen las coordenadas de la zona de aterrizaje y la velocidad límite de contacto con el suelo.

Las acciones disponibles para el agente son continuas, debido al parámetro `continuous = True` especificado al crear el entorno.
Estas acciones pueden incluir el empuje en dos direcciones (principal y lateral), permitiendo un control detallado y continuo sobre la nave lunar.

El agente recibe recompensas basadas en varios aspectos del aterrizaje: una recompensa positiva por aterrizar suavemente en la zona de aterrizaje penalizaciones por colisiones con obstáculos o aterrizajes bruscos y penalizaciones por el uso excesivo de combustible.

En comparación con el juego de Blackjack, donde las acciones son discretas (por ejemplo, pedir una carta, plantarse), en LunarLander las acciones son continuas. Esto significa que en cada paso de tiempo, el agente puede decidir exactamente cuánto empuje aplicar en la dirección principal y lateral, en lugar de tener opciones discretas predefinidas.

El parámetro `continuous = True` en el entorno LunarLander permite un control más preciso y realista sobre las acciones del agente, pero al mismo tiempo introduce desafíos adicionales debido a la naturaleza continua del espacio de acción.

#### **2.2.2 Generando un Baseline (0.2 puntos)**

* Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas.
* ¿Cómo calificaría el performance de esta política?

In [ ]:
env.step(env.action_space.sample())

(array([-0.01191082,  1.4102659 , -0.40151438, -0.01819595,  0.01363343,
         0.0904926 ,  0.        ,  0.        ], dtype=float32),
 -0.5964644918817215,
 False,
 False,
 {})

In [ ]:
# Función para ejecutar una simulación con acciones aleatorias
def run_random_simulation():
    total_reward = 0
    done = False
    state = env.reset()

    while not done:
        action = env.action_space.sample()  # Escoger una acción aleatoria
        next_state, reward, done, _, _ = env.step(action)  # Capturar solo los primeros tres valores relevantes
        total_reward += reward

    return total_reward

# Ejecutar la simulación 10 veces
num_simulations = 10
rewards = []

for i in range(num_simulations):
    reward = run_random_simulation()
    rewards.append(reward)
    print(f"Simulación {i+1}: Recompensa = {reward}")

# Calcular el promedio y la desviación estándar de las recompensas
avg_reward = np.mean(rewards)
std_reward = np.std(rewards)

print(f"\nPromedio de recompensas: {avg_reward}")
print(f"Desviación estándar de recompensas: {std_reward}")

Simulación 1: Recompensa = -170.03922427701326
Simulación 2: Recompensa = -277.7454075857138
Simulación 3: Recompensa = -453.6606372759719
Simulación 4: Recompensa = -70.46970276221907
Simulación 5: Recompensa = -301.45885465932326
Simulación 6: Recompensa = -91.27572898012953
Simulación 7: Recompensa = -286.39482683941645
Simulación 8: Recompensa = -48.72144751554576
Simulación 9: Recompensa = -56.45068565896193
Simulación 10: Recompensa = -303.52766365369735

Promedio de recompensas: -205.97441792079925
Desviación estándar de recompensas: 130.95624926928534


La política aleatoria no logra aprender o mantener una estrategia efectiva para aterrizar la nave lunar suavemente en la zona de aterrizaje. Esto se refleja en el promedio negativo de recompensas, lo cual indica que la mayoría de las simulaciones terminan con un desempeño subóptimo o fallido.

Por otro lado, la alta desviación estándar sugiere que la política aleatoria produce resultados inconsistentes. A veces puede obtener recompensas relativamente altas debido a la aleatoriedad de acciones que ocasionalmente pueden resultar en aterrizajes más suaves o menos penalizados. Sin embargo, estos episodios de buen desempeño son poco frecuentes y no son consistentes.

#### **2.2.3 Entrenamiento de modelo (0.2 puntos)**

* A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy

env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

# Configurar y entrenar el modelo SAC
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Guardar el modelo entrenado
model.save("sac_lunarlander")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 121      |
|    ep_rew_mean     | -312     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 35       |
|    time_elapsed    | 13       |
|    total_timesteps | 484      |
| train/             |          |
|    actor_loss      | 1.86     |
|    critic_loss     | 12       |
|    ent_coef        | 0.897    |
|    ent_coef_loss   | -0.263   |
|    learning_rate   | 0.0003   |
|    n_updates       | 383      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 124      |
|    ep_rew_mean     | -318     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 33       |
|    time_elapsed    | 29       |
|    total_timesteps | 993      |
| train/             |

#### **2.2.4 Evaluación de modelo (0.2 puntos)**

* Repita el ejercicio 2.2.2 pero utilizando el modelo entrenado.
* ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Evaluar el modelo entrenado
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

Mean reward: -30.33686969203573, Std reward: 107.65907603501222


El valor de -30.34 indica que, en promedio, el agente obtiene una recompensa negativa durante los episodios de evaluación. Esto sugiere que el agente no logra realizar aterrizajes exitosos de manera consistente y, en general, no cumple con el objetivo principal de aterrizar suavemente en la zona de aterrizaje. Pero de todas formas, este valor es más cercano a cero que el del baseline, por lo que es mejor (pero no óptimo).

Por otro lado, la alta desviación estándar de 107.66 indica una gran variabilidad en el desempeño del agente entre los diferentes episodios evaluados. Esto puede deberse a la naturaleza estocástica del entorno LunarLander y a la complejidad de aprender una política efectiva para controlar la nave lunar. De todas formas, esta es menor al baseline, por lo que es relativamente mejor.

#### **2.2.5 Optimización de modelo (0.2 puntos)**

* Repita los ejercicios 2.2.3 y 2.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
  - `total_timesteps`
  - `learning_rate`
  - `batch_size`

* Una vez optimizado el modelo, use la función `export_gif` entregada para estudiar el comportamiento de su agente en la resolución del ambiente, comente sobre sus resultados.

* Adjunte el gif generado en su entrega. Si, además, adjuntan el gif en el markdown tendrán un bonus de 0.1.

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy


# Entrenamiento
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True)
total_timesteps = 20000
learning_rate = 0.0005
batch_size = 256
model = SAC("MlpPolicy", env, learning_rate=learning_rate, batch_size=batch_size, verbose=1)
model.learn(total_timesteps=total_timesteps)

# Evaluación
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")
env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.2     |
|    ep_rew_mean     | -242     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 98       |
|    time_elapsed    | 3        |
|    total_timesteps | 361      |
| train/             |          |
|    actor_loss      | 1.3      |
|    critic_loss     | 43.2     |
|    ent_coef        | 0.885    |
|    ent_coef_loss   | -0.309   |
|    learning_rate   | 0.0005   |
|    n_updates       | 260      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 115      |
|    ep_rew_mean     | -244     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 92       |
|    time_elapsed    | 9        |
|    total_timesteps | 923      |
| train/             

In [ ]:
model.save("best_sac_lunarlander")

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
export_gif(model)